# QS2021 USA Ranking 分析

## 爬虫

In [99]:
import pandas as pd
import requests
import json
import re

In [100]:
uni_name = "uni"
Employability = "ind_27"
Learning_Experience = "ind_2054337"
Diversity_Internationalisation = "ind_2054338"
Research = "ind_2054339"
one, two, three, four, five = [],[],[],[],[]
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36"}

In [101]:
response = requests.get("https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/3738856_indicators.txt?1620211931?v=1620413640389",headers=headers,timeout=5)
qs_usa_dataset = json.loads(response.text)["data"]
for i in qs_usa_dataset:
    one.append(re.match('.*class="uni-link">(.*)</a></div></div>', i[uni_name]).group(1))
    two.append(float(re.match('.*class="td-wrap-in">(.*)</div></div>', i[Employability]).group(1)))
    three.append(float(re.match('.*class="td-wrap-in">(.*)</div></div>', i[Learning_Experience]).group(1)))
    four.append(float(re.match('.*class="td-wrap-in">(.*)</div></div>', i[Diversity_Internationalisation]).group(1)))
    five.append(float(re.match('.*class="td-wrap-in">(.*)</div></div>', i[Research]).group(1)))

In [102]:
df = pd.DataFrame({
    "name":one,
    "Employability":two,
    "Learning_Experience":three,
    "Diversity_Inter":four,
    "Research":five}
)

In [103]:
df.index +=1

In [104]:
df

,name,Employability,Learning_Experience,Diversity_Inter,Research
1,Harvard University,100.0,96.8,76.0,100.0
2,Stanford University,83.4,96.8,82.0,93.5
3,Massachusetts Institute of Technology (MIT),75.4,96.8,80.0,90.6
4,"University of California, Berkeley (UCB)",57.9,81.4,78.0,91.7
5,"University of California, Los Angeles (UCLA)",57.7,90.5,84.0,78.7
...,...,...,...,...,...
348,University of the Incarnate Word,7.3,49.5,64.0,20.4
349,Utah State University,13.2,46.4,34.0,45.0
350,Waynesburg University,7.3,63.2,64.0,10.5
351,Wichita State University,11.1,43.2,54.0,32.4


## 参数调整

### 就业50% ，学习经历50%

In [105]:
df2 = df.copy(deep=True)

In [106]:
df2["Scores"] = df2["Employability"]*0.5 + df2["Learning_Experience"]*0.5
df2 = df2.sort_values("Scores",ascending=False)
df2 = df2.reset_index()
df2.index +=1
df2.head(20)

,index,name,Employability,Learning_Experience,Diversity_Inter,Research,Scores
1,1,Harvard University,100.0,96.8,76.0,100.0,98.40
2,2,Stanford University,83.4,96.8,82.0,93.5,90.10
3,3,Massachusetts Institute of Technology (MIT),75.4,96.8,80.0,90.6,86.10
4,6,Yale University,66.6,96.8,70.0,71.2,81.70
5,10,University of Pennsylvania,61.5,96.8,72.0,66.0,79.15
6,8,Princeton University,58.1,100.0,70.0,71.5,79.05
7,7,Columbia University,60.6,96.8,70.0,75.2,78.70
8,13,Duke University,49.8,100.0,70.0,59.9,74.90
9,5,"University of California, Los Angeles (UCLA)",57.7,90.5,84.0,78.7,74.10
10,9,New York University (NYU),54.5,92.3,86.0,63.7,73.40


In [107]:
df2[(df2["name"] == "University of California, Los Angeles (UCLA)")| 
   (df2["name"] == "New York University (NYU)")| 
   (df2["name"] == "California Institute of Technology (Caltech)")]

,index,name,Employability,Learning_Experience,Diversity_Inter,Research,Scores
9,5,"University of California, Los Angeles (UCLA)",57.7,90.5,84.0,78.7,74.10
10,9,New York University (NYU),54.5,92.3,86.0,63.7,73.40
16,21,California Institute of Technology (Caltech),39.9,100.0,56.0,63.1,69.95


### 学习经历50%，科研50%

In [108]:
df3 = df.copy(deep=True)

In [109]:
df3["Scores"] = df3["Learning_Experience"]*0.5 + df3["Research"]*0.5
df3 = df3.sort_values("Scores",ascending=False)
df3 = df3.reset_index()
df3.index +=1
df3.head(20)

,index,name,Employability,Learning_Experience,Diversity_Inter,Research,Scores
1,1,Harvard University,100.0,96.8,76.0,100.0,98.40
2,2,Stanford University,83.4,96.8,82.0,93.5,95.15
3,3,Massachusetts Institute of Technology (MIT),75.4,96.8,80.0,90.6,93.70
4,4,"University of California, Berkeley (UCB)",57.9,81.4,78.0,91.7,86.55
5,7,Columbia University,60.6,96.8,70.0,75.2,86.00
6,8,Princeton University,58.1,100.0,70.0,71.5,85.75
7,11,University of Chicago,46.6,100.0,68.0,70.0,85.00
8,5,"University of California, Los Angeles (UCLA)",57.7,90.5,84.0,78.7,84.60
9,6,Yale University,66.6,96.8,70.0,71.2,84.00
10,21,California Institute of Technology (Caltech),39.9,100.0,56.0,63.1,81.55


In [110]:
df3[(df3["name"] == "University of California, Los Angeles (UCLA)")| 
   (df3["name"] == "New York University (NYU)")| 
   (df3["name"] == "California Institute of Technology (Caltech)")]

,index,name,Employability,Learning_Experience,Diversity_Inter,Research,Scores
8,5,"University of California, Los Angeles (UCLA)",57.7,90.5,84.0,78.7,84.60
10,21,California Institute of Technology (Caltech),39.9,100.0,56.0,63.1,81.55
17,9,New York University (NYU),54.5,92.3,86.0,63.7,78.00


### 就业能力，学习经历，科研各1/3

In [111]:
df4 = df.copy(deep=True)
df4["Scores"] = df4["Employability"]/3 + df4["Learning_Experience"]/3 + df4["Research"]/3
df4 = df4.sort_values("Scores",ascending=False)
df4 = df4.reset_index()
df4.index +=1
df4.head(20)

,index,name,Employability,Learning_Experience,Diversity_Inter,Research,Scores
1,1,Harvard University,100.0,96.8,76.0,100.0,98.933333
2,2,Stanford University,83.4,96.8,82.0,93.5,91.233333
3,3,Massachusetts Institute of Technology (MIT),75.4,96.8,80.0,90.6,87.600000
4,6,Yale University,66.6,96.8,70.0,71.2,78.200000
5,7,Columbia University,60.6,96.8,70.0,75.2,77.533333
6,4,"University of California, Berkeley (UCB)",57.9,81.4,78.0,91.7,77.000000
7,8,Princeton University,58.1,100.0,70.0,71.5,76.533333
8,5,"University of California, Los Angeles (UCLA)",57.7,90.5,84.0,78.7,75.633333
9,10,University of Pennsylvania,61.5,96.8,72.0,66.0,74.766667
10,11,University of Chicago,46.6,100.0,68.0,70.0,72.200000


In [112]:
df4[(df4["name"] == "University of California, Los Angeles (UCLA)")| 
   (df4["name"] == "New York University (NYU)")| 
   (df4["name"] == "California Institute of Technology (Caltech)")]

,index,name,Employability,Learning_Experience,Diversity_Inter,Research,Scores
8,5,"University of California, Los Angeles (UCLA)",57.7,90.5,84.0,78.7,75.633333
11,9,New York University (NYU),54.5,92.3,86.0,63.7,70.166667
15,21,California Institute of Technology (Caltech),39.9,100.0,56.0,63.1,67.666667
